# Part I: Foundations of FastAPI

## Chapter 3: Automatic Documentation

One of FastAPI's most powerful features is its automatic API documentation generation. As you define your endpoints, FastAPI simultaneously creates a complete OpenAPI specification and interactive documentation interfaces. This chapter explores how to leverage, customize, and extend these documentation capabilities to create professional, informative API references.

---

### 3.1 Swagger UI: Interactive API Exploration

**Swagger UI** is the default interactive documentation interface that comes with FastAPI. It provides a visual, browser-based interface where developers can explore your API endpoints, understand their parameters, and even test requests directly.

#### Accessing Swagger UI

By default, Swagger UI is available at the `/docs` path:

```python
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def get_item(item_id: int, q: str | None = None):
    """Retrieve an item by its ID."""
    return {"item_id": item_id, "q": q}
```

Navigate to `http://localhost:8000/docs` in your browser:

```
┌─────────────────────────────────────────────────────────────┐
│                      Swagger UI                              │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│   My First API                                    v0.1.0     │
│   A simple API to learn FastAPI fundamentals                 │
│                                                              │
│   ┌─────────────────────────────────────────────────────┐   │
│   │ GET  /items/{item_id}    Retrieve an item by ID     │   │
│   │                                                       │   │
│   │ [Try it out]  [Cancel]                               │   │
│   │                                                       │   │
│   │ item_id *  [___________]                             │   │
│   │ q          [___________]                             │   │
│   │                                                       │   │
│   │ [Execute]                                             │   │
│   └─────────────────────────────────────────────────────┘   │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

#### Swagger UI Features

**1. Interactive Testing**

The "Try it out" button allows users to:
- Input parameter values
- Modify request headers
- Add request bodies
- Execute real API calls
- View actual responses

**2. Schema Documentation**

Each endpoint displays:
- Required vs optional parameters
- Parameter types and constraints
- Request body schemas with examples
- Response schemas for different status codes

**3. Response Examples**

After executing a request, Swagger UI shows:
- Server response body
- Response headers
- HTTP status code
- Response time

#### Enabling and Disabling Swagger UI

You can control Swagger UI availability during application creation:

```python
from fastapi import FastAPI

# Disable docs entirely
app = FastAPI(docs_url=None)

# Or use a custom path
app = FastAPI(docs_url="/api/docs")

# Default: /docs
app = FastAPI(docs_url="/docs")
```

#### Customizing Swagger UI Appearance

Use `swagger_ui_parameters` to modify Swagger UI's behavior:

```python
app = FastAPI(
    swagger_ui_parameters={
        "syntaxHighlight.theme": "monokai",  # Syntax highlighting theme
        "deepLinking": True,  # Enable deep linking to operations
        "displayOperationId": True,  # Show operation IDs
        "filter": True,  # Enable search/filter bar
        "showExtensions": True,  # Show vendor extensions
        "showCommonExtensions": True,
        "persistAuthorization": True,  # Remember authorization between reloads
        "displayRequestDuration": True,  # Show request duration
        "docExpansion": "list",  # Default expansion: "list", "full", or "none"
        "defaultModelsExpandDepth": 1,  # How deep to expand models
        "defaultModelExpandDepth": 1,
    }
)
```

**Common Customization Options:**

| Parameter | Values | Effect |
|-----------|--------|--------|
| `docExpansion` | `"list"`, `"full"`, `"none"` | Controls default expansion of operations |
| `defaultModelsExpandDepth` | integer (default: 1) | Model expansion depth; -1 hides all |
| `displayOperationId` | boolean | Shows operation ID alongside path |
| `filter` | boolean | Shows search bar to filter endpoints |
| `persistAuthorization` | boolean | Preserves authorization data across page reloads |
| `syntaxHighlight.theme` | string | Code highlighting theme |
| `tryItOutEnabled` | boolean | Enables "Try it out" by default |

#### Adding Authorization to Swagger UI

Swagger UI supports interactive authentication. Define security schemes to enable the "Authorize" button:

```python
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer

app = FastAPI()

# Define the OAuth2 security scheme
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")


@app.get("/users/me")
async def read_users_me(token: str = Depends(oauth2_scheme)):
    """
    Protected endpoint that requires authentication.

    In Swagger UI, click the 'Authorize' button to enter a token.
    """
    return {"token": token}
```

This adds an "Authorize" button to Swagger UI where users can input their bearer token, which will be included in all subsequent requests.

---

### 3.2 ReDoc: Alternative Documentation

**ReDoc** is an alternative documentation interface that provides a cleaner, more polished look. While Swagger UI is optimized for interactivity and testing, ReDoc excels at readability and is often preferred for public-facing documentation.

#### Accessing ReDoc

By default, ReDoc is available at `/redoc`:

```python
from fastapi import FastAPI

app = FastAPI()
# ReDoc available at http://localhost:8000/redoc
```

#### ReDoc Features

**1. Three-Panel Layout**

ReDoc organizes documentation into three columns:
- **Left**: Navigation panel with endpoints grouped by tags
- **Center**: Main content area with endpoint details
- **Right**: Request/response schema reference

**2. Clean, Readable Design**

ReDoc emphasizes:
- Typography and spacing for readability
- Color-coded HTTP methods
- Expandable schemas with type information
- Request/response examples

**3. Code Samples**

ReDoc can display code samples in multiple languages (requires configuration):

```text
┌─────────────────────────────────────────────────────────────┐
│                       ReDoc Layout                          │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  ┌─────────┐  ┌──────────────────────┐  ┌────────────────┐  │
│  │ ENDPOINTS│  │    ENDPOINT DETAILS   │  │    SCHEMA     │  │
│  │         │  │                       │  │               │  │
│  │ ▼ Users │  │  GET /users/{id}     │  │  User         │  │
│  │   GET   │  │  ─────────────────   │  │  ───────────  │  │
│  │   /users│  │                       │  │  id: integer  │  │
│  │   POST  │  │  Retrieve a user by  │  │  name: string │  │
│  │   /users│  │  their unique ID.     │  │  email: string│  │
│  │         │  │                       │  │               │  │
│  │ ▼ Items │  │  Parameters:          │  │               │  │
│  │   GET   │  │  • id (path)          │  │               │  │
│  │   /items│  │                       │  │               │  │
│  │         │  │  Responses:           │  │               │  │
│  │         │  │  • 200: User object   │  │               │  │
│  │         │  │  • 404: Not found     │  │               │  │
│  └─────────┘  └──────────────────────┘  └────────────────┘  │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

#### Configuring ReDoc

Control ReDoc availability and customization:

```python
from fastapi import FastAPI

app = FastAPI(
    redoc_url="/api/docs",  # Custom path for ReDoc
    # redoc_url=None,       # Disable ReDoc entirely
)
```

For advanced ReDoc configuration, you can use a custom JavaScript setup (covered in advanced documentation customization).

#### When to Use ReDoc vs Swagger UI

| Use Swagger UI When | Use ReDoc When |
|---------------------|----------------|
| Developers need to test endpoints | Documentation is for external consumers |
| Interactive "Try it out" is important | Clean, readable presentation matters |
| Internal APIs for development teams | Public APIs with external users |
| Rapid prototyping and debugging | Formal API reference documentation |

---

> **Industry Standard:** Many teams use both: Swagger UI (`/docs`) for internal development and testing, and ReDoc (`/redoc`) as the public-facing documentation. Some even disable Swagger UI in production for security.

---

### 3.3 OpenAPI Specification: Understanding the JSON Schema

At the core of FastAPI's documentation is the **OpenAPI Specification (OAS)**, a standard format for describing REST APIs. Understanding OpenAPI helps you leverage FastAPI's full documentation potential.

#### What is OpenAPI?

OpenAPI (formerly Swagger Specification) is a standard, language-agnostic interface description for REST APIs. It allows both humans and computers to:
- Discover and understand API capabilities
- Generate client libraries in various languages
- Create mock servers for testing
- Validate API implementations against specifications

#### Accessing the OpenAPI Schema

FastAPI generates your OpenAPI schema at `/openapi.json` by default:

```python
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def get_item(item_id: int):
    return {"item_id": item_id}
```

Navigate to `http://localhost:8000/openapi.json`:

```json
{
  "openapi": "3.1.0",
  "info": {
    "title": "FastAPI",
    "version": "0.1.0"
  },
  "paths": {
    "/items/{item_id}": {
      "get": {
        "summary": "Get Item",
        "operationId": "get_item_items__item_id__get",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    }
  }
}
```

#### OpenAPI Structure Overview

The OpenAPI specification consists of several key sections:

**1. OpenAPI Version**

```json
"openapi": "3.1.0"
```

FastAPI uses OpenAPI 3.1.0 by default, which supports the full JSON Schema specification.

**2. Info Object**

```json
"info": {
  "title": "My API",
  "description": "A comprehensive API for...",
  "version": "1.0.0",
  "contact": {
    "name": "API Support",
    "email": "support@example.com"
  },
  "license": {
    "name": "MIT",
    "url": "https://opensource.org/licenses/MIT"
  }
}
```

**3. Servers**

```json
"servers": [
  {
    "url": "https://api.example.com/v1",
    "description": "Production server"
  },
  {
    "url": "http://localhost:8000",
    "description": "Development server"
  }
]
```

**4. Paths**

The `paths` object contains all API endpoints:

```json
"paths": {
  "/users": {
    "get": {
      "tags": ["users"],
      "summary": "List all users",
      "operationId": "list_users_users_get",
      "parameters": [...],
      "responses": {...}
    },
    "post": {...}
  }
}
```

**5. Components**

Reusable schemas, parameters, and security schemes:

```json
"components": {
  "schemas": {
    "User": {
      "type": "object",
      "properties": {
        "id": {"type": "integer"},
        "name": {"type": "string"}
      }
    }
  },
  "securitySchemes": {
    "bearerAuth": {
      "type": "http",
      "scheme": "bearer"
    }
  }
}
```

#### Customizing the OpenAPI Schema

Customize metadata in your FastAPI application:

```python
from fastapi import FastAPI

app = FastAPI(
    title="E-Commerce API",
    description="""
## Overview

This API provides endpoints for managing an e-commerce platform.

## Features

* **User Management**: Registration, authentication, profiles
* **Product Catalog**: Browse and search products
* **Shopping Cart**: Add/remove items, checkout
* **Order Processing**: Track and manage orders

## Authentication

Most endpoints require a Bearer token. Use the `/auth/login` endpoint to obtain one.
    """,
    version="2.1.0",
    terms_of_service="https://example.com/terms/",
    contact={
        "name": "API Support Team",
        "url": "https://example.com/support",
        "email": "api-support@example.com",
    },
    license_info={
        "name": "Apache 2.0",
        "url": "https://www.apache.org/licenses/LICENSE-2.0.html",
    },
    openapi_url="/api/openapi.json",  # Custom path for the schema
    docs_url="/api/docs",  # Custom Swagger UI path
    redoc_url="/api/redoc",  # Custom ReDoc path
)
```

#### Adding Servers to OpenAPI

Define server URLs that appear in documentation:

```python
app = FastAPI(
    servers=[
        {
            "url": "http://localhost:8000",
            "description": "Development server",
        },
        {
            "url": "https://staging-api.example.com",
            "description": "Staging server",
        },
        {
            "url": "https://api.example.com",
            "description": "Production server",
        },
    ]
)
```

This creates a server selection dropdown in Swagger UI:

```
┌─────────────────────────────────────────────────────┐
│ Servers: [Production server ▼]                      │
│          ├─ Development server                      │
│          ├─ Staging server                          │
│          └─ Production server                       │
└─────────────────────────────────────────────────────┘
```

#### Programmatic Access to OpenAPI Schema

You can access and modify the OpenAPI schema programmatically:

```python
from fastapi import FastAPI
from fastapi.openapi.utils import get_openapi

app = FastAPI()


def custom_openapi():
    """
    Generate a customized OpenAPI schema.
    This function is called once and the result is cached.
    """
    if app.openapi_schema:
        return app.openapi_schema

    openapi_schema = get_openapi(
        title="Custom API",
        version="1.0.0",
        description="This is a custom OpenAPI schema",
        routes=app.routes,
    )

    # Add custom extensions
    openapi_schema["info"]["x-logo"] = {
        "url": "https://example.com/logo.png"
    }

    # Add security globally
    openapi_schema["security"] = [{"bearerAuth": []}]

    app.openapi_schema = openapi_schema
    return app.openapi_schema


# Override the default OpenAPI generation
app.openapi = custom_openapi
```

### 3.4 Customizing Docs: Hiding Endpoints, Renaming, and Organizing Tags

As your API grows, organizing and customizing documentation becomes essential. FastAPI provides several mechanisms for controlling how endpoints appear in your documentation.

#### Organizing with Tags

**Tags** group related endpoints in Swagger UI and ReDoc, creating a navigable structure:

```python
from fastapi import FastAPI

app = FastAPI()


@app.get("/users", tags=["users"])
async def list_users():
    """List all users."""
    return []


@app.post("/users", tags=["users"])
async def create_user():
    """Create a new user."""
    return {"id": 1}


@app.get("/items", tags=["items"])
async def list_items():
    """List all items."""
    return []


@app.post("/items", tags=["items"])
async def create_item():
    """Create a new item."""
    return {"id": 1}
```

In Swagger UI, endpoints are grouped under expandable sections:

```
▼ users
    GET  /users    List all users
    POST /users    Create a new user

▼ items
    GET  /items    List all items
    POST /items    Create a new item
```

#### Tag Metadata and Descriptions

Add detailed metadata for tags using `openapi_tags`:

```python
from fastapi import FastAPI

tags_metadata = [
    {
        "name": "users",
        "description": "Operations with users. The **login** logic is also here.",
    },
    {
        "name": "items",
        "description": "Manage items. Includes CRUD operations for the item catalog.",
        "externalDocs": {
            "description": "Items external documentation",
            "url": "https://example.com/docs/items",
        },
    },
    {
        "name": "admin",
        "description": "Administrative operations. Requires elevated permissions.",
    },
]

app = FastAPI(openapi_tags=tags_metadata)


@app.get("/users", tags=["users"])
async def list_users():
    return []


@app.get("/items", tags=["items"])
async def list_items():
    return []


@app.delete("/cache", tags=["admin"])
async def clear_cache():
    return {"message": "Cache cleared"}
```

Tag metadata supports:
- `name`: The tag identifier (matches `tags=` in decorators)
- `description`: Detailed description (supports Markdown)
- `externalDocs`: Link to external documentation

#### Multiple Tags per Endpoint

An endpoint can have multiple tags:

```python
@app.get("/users/{user_id}/items", tags=["users", "items"])
async def get_user_items(user_id: int):
    """Get all items belonging to a user.

    This endpoint appears under both 'users' and 'items' sections.
    """
    return {"user_id": user_id, "items": []}
```

#### Hiding Endpoints from Documentation

Sometimes you need endpoints that shouldn't appear in public documentation:

**Method 1: Using `include_in_schema`**

```python
from fastapi import FastAPI

app = FastAPI()


@app.get("/internal/health", include_in_schema=False)
async def internal_health_check():
    """
    Internal health check endpoint.

    This won't appear in /docs or /redoc, but is still accessible.
    """
    return {"status": "healthy"}


@app.get("/internal/metrics", include_in_schema=False)
async def internal_metrics():
    """Endpoint for Prometheus metrics scraping."""
    return {"metrics": {}}
```

**Method 2: Using a separate router for internal endpoints**

```python
from fastapi import FastAPI, APIRouter

app = FastAPI()

# Public router - included in docs
public_router = APIRouter()


@public_router.get("/users")
async def list_users():
    return []


# Internal router - not included in docs
internal_router = APIRouter(include_in_schema=False)


@internal_router.get("/health")
async def health():
    return {"status": "ok"}


# Include routers
app.include_router(public_router)
app.include_router(internal_router, prefix="/internal")
```

#### Renaming Endpoints

Control the operation ID and summary displayed in documentation:

```python
from fastapi import FastAPI

app = FastAPI()


@app.get(
    "/users/{user_id}",
    name="get_user_by_id",  # Custom operation ID
    summary="Retrieve User",  # Short title
    description="""
Retrieve a specific user by their unique identifier.

## Business Logic

1. Validates the user ID format
2. Queries the database for the user
3. Returns user profile or 404 if not found

## Error Codes

* `404`: User not found
* `400`: Invalid user ID format
    """,
)
async def get_user(user_id: int):
    return {"user_id": user_id}
```

#### Deprecating Endpoints

Mark endpoints as deprecated without removing them:

```python
@app.get(
    "/legacy/users",
    deprecated=True,
    tags=["users"],
    summary="[DEPRECATED] List Users (Old Version)",
    description="This endpoint is deprecated. Use `GET /users` instead.",
)
async def legacy_list_users():
    """
    Legacy endpoint for backward compatibility.

    Will be removed in version 3.0.0.
    """
    return []
```

Deprecated endpoints appear with a strikethrough in Swagger UI:

```
~~GET /legacy/users    [DEPRECATED] List Users (Old Version)~~
```

#### Adding Examples to Models and Parameters

Examples make documentation more helpful:

```python
from typing import Annotated

from fastapi import FastAPI, Body, Query
from pydantic import BaseModel, Field

app = FastAPI()


class Item(BaseModel):
    name: str = Field(examples=["Laptop"])
    price: float = Field(examples=[999.99])
    description: str | None = Field(default=None, examples=["A powerful laptop"])

    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "Gaming Laptop",
                    "price": 1499.99,
                    "description": "High-performance laptop for gaming",
                },
                {
                    "name": "Office Chair",
                    "price": 299.99,
                    "description": "Ergonomic office chair",
                },
            ]
        }
    }


@app.post("/items")
async def create_item(item: Item):
    return item


@app.get("/search")
async def search(
    q: Annotated[
        str,
        Query(
            description="Search query string",
            examples=["laptop", "chair", "monitor"],
        ),
    ],
):
    return {"query": q}
```

#### Using the `responses` Parameter

Document multiple response types:

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()


class ErrorDetail(BaseModel):
    code: str
    message: str


@app.get(
    "/items/{item_id}",
    responses={
        200: {
            "description": "Successful Response",
            "content": {
                "application/json": {
                    "example": {"id": 1, "name": "Laptop", "price": 999.99}
                }
            },
        },
        404: {
            "model": ErrorDetail,
            "description": "Item not found",
            "content": {
                "application/json": {
                    "example": {"code": "NOT_FOUND", "message": "Item 999 not found"}
                }
            },
        },
        400: {
            "model": ErrorDetail,
            "description": "Invalid item ID",
        },
    },
)
async def get_item(item_id: int):
    if item_id < 1:
        raise HTTPException(
            status_code=400,
            detail={"code": "INVALID_ID", "message": "Item ID must be positive"},
        )
    if item_id > 100:
        raise HTTPException(
            status_code=404,
            detail={"code": "NOT_FOUND", "message": f"Item {item_id} not found"},
        )
    return {"id": item_id, "name": f"Item {item_id}", "price": 99.99}
```

#### Complete Documentation Example

Here's a comprehensive example combining all documentation features:

```python
# main.py
from typing import Annotated

from fastapi import FastAPI, HTTPException, Path, Query, Body
from pydantic import BaseModel, Field, EmailStr

# Tag metadata for organization
tags_metadata = [
    {
        "name": "users",
        "description": "User management operations. Create, read, update, and delete users.",
    },
    {
        "name": "products",
        "description": "Product catalog operations.",
        "externalDocs": {
            "description": "Products Guide",
            "url": "https://example.com/docs/products",
        },
    },
    {
        "name": "orders",
        "description": "Order processing and tracking.",
    },
]

app = FastAPI(
    title="E-Commerce API",
    description="""
## Welcome to the E-Commerce API

This API provides comprehensive endpoints for managing an online store.

### Key Features

- 🛒 Product catalog management
- 👥 User authentication and profiles
- 📦 Order processing and tracking
- 💳 Payment integration

### Getting Started

1. Register a user account via `POST /users`
2. Login to get an authentication token
3. Use the token in the `Authorization` header for protected endpoints

### Rate Limiting

- Free tier: 100 requests/hour
- Pro tier: 1000 requests/hour
    """,
    version="1.0.0",
    openapi_tags=tags_metadata,
    contact={
        "name": "API Support",
        "email": "api@example.com",
        "url": "https://example.com/support",
    },
    license_info={
        "name": "MIT License",
        "url": "https://opensource.org/licenses/MIT",
    },
    servers=[
        {"url": "http://localhost:8000", "description": "Development"},
        {"url": "https://api.example.com", "description": "Production"},
    ],
)


# Models
class UserCreate(BaseModel):
    """Model for creating a new user."""

    username: str = Field(
        min_length=3,
        max_length=50,
        pattern=r"^[a-zA-Z0-9_]+$",
        examples=["alice_dev"],
    )
    email: EmailStr = Field(examples=["alice@example.com"])
    password: str = Field(min_length=8, examples=["securepassword123"])


class UserResponse(BaseModel):
    """Model for user data in responses."""

    id: int
    username: str
    email: str
    is_active: bool = True


class Product(BaseModel):
    """Model for a product in the catalog."""

    id: int
    name: str = Field(examples=["Wireless Mouse"])
    price: float = Field(gt=0, examples=[29.99])
    stock: int = Field(ge=0, examples=[150])
    category: str = Field(examples=["Electronics"])


class ErrorDetail(BaseModel):
    """Model for error responses."""

    code: str
    message: str
    details: dict | None = None


# User Endpoints
@app.post(
    "/users",
    response_model=UserResponse,
    status_code=201,
    tags=["users"],
    summary="Create a New User",
    description="""
Register a new user account.

The username must be unique and contain only alphanumeric characters and underscores.
    """,
    responses={
        201: {"description": "User created successfully"},
        400: {
            "model": ErrorDetail,
            "description": "Username already exists",
        },
    },
)
async def create_user(user: UserCreate):
    """Create a new user account."""
    return {
        "id": 1,
        "username": user.username,
        "email": user.email,
        "is_active": True,
    }


@app.get(
    "/users/{user_id}",
    response_model=UserResponse,
    tags=["users"],
    responses={
        404: {"model": ErrorDetail, "description": "User not found"},
    },
)
async def get_user(
    user_id: Annotated[int, Path(ge=1, description="The unique user ID")],
):
    """Retrieve a user by their ID."""
    if user_id > 1000:
        raise HTTPException(
            status_code=404,
            detail={"code": "NOT_FOUND", "message": f"User {user_id} not found"},
        )
    return {"id": user_id, "username": "alice", "email": "alice@example.com"}


# Product Endpoints
@app.get(
    "/products",
    response_model=list[Product],
    tags=["products"],
    summary="List Products",
)
async def list_products(
    category: Annotated[str | None, Query(description="Filter by category")] = None,
    min_price: Annotated[float | None, Query(ge=0)] = None,
    max_price: Annotated[float | None, Query(ge=0)] = None,
    skip: Annotated[int, Query(ge=0)] = 0,
    limit: Annotated[int, Query(ge=1, le=100)] = 10,
):
    """
    List products with optional filtering.

    Supports filtering by category and price range.
    Results are paginated with skip and limit parameters.
    """
    products = [
        {"id": 1, "name": "Laptop", "price": 999.99, "stock": 50, "category": "Electronics"},
        {"id": 2, "name": "Mouse", "price": 29.99, "stock": 200, "category": "Electronics"},
        {"id": 3, "name": "Desk", "price": 199.99, "stock": 30, "category": "Furniture"},
    ]
    return products[skip : skip + limit]


@app.get(
    "/products/{product_id}",
    response_model=Product,
    tags=["products"],
    responses={404: {"model": ErrorDetail}},
)
async def get_product(product_id: int = Path(ge=1)):
    """Retrieve detailed information about a specific product."""
    if product_id > 100:
        raise HTTPException(
            status_code=404,
            detail={"code": "NOT_FOUND", "message": f"Product {product_id} not found"},
        )
    return {
        "id": product_id,
        "name": "Sample Product",
        "price": 99.99,
        "stock": 100,
        "category": "General",
    }


# Hidden internal endpoint
@app.get("/internal/health", include_in_schema=False)
async def health_check():
    """Internal health check (not shown in docs)."""
    return {"status": "healthy"}


# Deprecated endpoint
@app.get(
    "/v1/products",
    tags=["products"],
    deprecated=True,
    summary="[DEPRECATED] Old Product List",
)
async def legacy_list_products():
    """Legacy endpoint. Use GET /products instead."""
    return []
```

---

### Summary

In this chapter, you've mastered FastAPI's automatic documentation:

1. **Swagger UI**: Interactive API exploration at `/docs` with customization options for appearance and behavior.

2. **ReDoc**: Clean, readable alternative documentation at `/redoc`, ideal for public-facing API references.

3. **OpenAPI Specification**: Understanding the underlying JSON schema that powers all documentation, including its structure and customization.

4. **Documentation Customization**: Organizing endpoints with tags, adding metadata, hiding sensitive endpoints, deprecating old versions, and providing comprehensive examples.

---

### Exercises

1. **Documentation Setup**: Create a FastAPI application with:
   - A custom title, description, and version
   - At least two servers (development and production)
   - Contact information and license

2. **Tag Organization**: Create an API with at least three tag groups (`users`, `products`, `orders`), each containing multiple endpoints. Add descriptive metadata to each tag.

3. **Response Documentation**: Create an endpoint that documents at least four different response codes (200, 201, 400, 404) with appropriate models and examples for each.

4. **Hidden Endpoints**: Create an API with:
   - Public endpoints visible in documentation
   - Internal health check endpoint hidden from docs
   - A deprecated endpoint clearly marked as such

5. **Complete API Documentation**: Build a small "Book Library" API with complete documentation including:
   - Tag organization
   - Model examples
   - Multiple response types
   - Deprecation notices where appropriate

---

### What's Next?

**Chapter 4: Deep Dive into Pydantic Models** will take you into the heart of data validation in FastAPI:
- Model structure and field definitions
- Field validation with `Field()` constraints
- Custom validators using `@field_validator`
- Nested models for complex data structures
- Special types like `EmailStr`, `HttpUrl`, and `UUID`
- Model configuration and advanced serialization



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='2. routing_and_basic_path_operations.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../2. data_validation_and_serialization_with_pydantic/4. deep_dive_into_pydantic_models.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
